<h1> GENERATE SIGNATURE <h1>

In [1]:
from sign import SignatureGenerator

In [2]:
types = {
    "Bid": [
      {"name": "swapId", "type": "uint256"},
      {"name": "nonce", "type": "uint256"},
      {"name": "signerWallet", "type": "address"},
      {"name": "sellAmount", "type": "uint256"},
      {"name": "buyAmount", "type": "uint256"},
      {"name": "referrer", "type": "address"},
    ]
  }

good_input = {
  "swapId": 1,
  "nonce": 1,
  "signerWallet": '0x28b2732460D09a8CDdd66cE956bE41D35B12F3b1',
  "sellAmount": '6000000',
  "buyAmount": '1000000000000000000',
  "referrer": '0x0000000000000000000000000000000000000000'
}

bad_input = {
  "swapId": 1,
  "nonce": 1,
  "signerWallet": '0xF49e9fc4487E1B23F92bC8effde5010e91b706eF',
  "sellAmount": '1000',
  "buyAmount": '1000000000000000',
  "referrer": '0x0000000000000000000000000000000000000000',
}

domain = {
  "name": 'RIBBON SWAP',
  "version": '1',
  "chainId": 42,
  "verifyingContract": '0x58848824baEb9678847aF487CB02EAba782FECB5'
}

privKey = "0xeb07e493c6eec28545bae35733c129461819c9e44be08dca3500bc405c232d8a"


In [3]:
signer = SignatureGenerator(privKey)
bid = signer.sign(domain, types, good_input)
bid

{'swapId': 1,
 'nonce': 1,
 'signerWallet': '0x28b2732460D09a8CDdd66cE956bE41D35B12F3b1',
 'sellAmount': '6000000',
 'buyAmount': '1000000000000000000',
 'referrer': '0x0000000000000000000000000000000000000000',
 'v': 27,
 'r': '0xd48860fab24673d45a03d58428f36bd7d62ac115972bd2a94e040503415a9478',
 's': '0x32eed933d6532dc613e3167a5e839bce2c1d577b3c4b2c73eea7411fec1c9a53'}

<h1> GET OTOKEN DETAILS <h1>

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

rpc = os.environ.get("RPC")

In [5]:
from option import oTokenFactory

In [6]:
oTokenAddress = "0xc68125707dd556975ee9d266E1844624f3128e77"
oToken = oTokenFactory(rpc, oTokenAddress)
oToken.getOtokenDetails()

{'collateralAsset': '0x0000000000000000000000000000000000000000',
 'underlyingAsset': '0x0000000000000000000000000000000000000000',
 'strikeAsset': '0x0000000000000000000000000000000000000000',
 'strikePrice': 0,
 'expiryTimestamp': 0,
 'isPut': False}

<h1> VALIDATE BID <h1>

In [7]:
from swap import SwapFactory

In [8]:
params = {"bid": bid}

In [9]:
swapAddress = "0x58848824baEb9678847aF487CB02EAba782FECB5"
swap = SwapFactory(rpc, swapAddress)
result = swap.validateBid(bid)
result

{'errors': 0}

In [10]:
bid = signer.sign(domain, types, bad_input)
result = swap.validateBid(bid)
result

{'errors': 5,
 'messages': ['BID_TOO_SMALL',
  'PRICE_TOO_LOW',
  'SIGNER_ALLOWANCE_LOW',
  'SIGNER_BALANCE_LOW']}